# Core

> The building blocks to the UI

## Imports

In [ ]:
#| default_exp core

In [ ]:
#| hide
#| export
import fasthtml.common as fh
from fasthtml.svg import Svg
from enum import Enum
from fasthtml.components import Uk_select,Uk_input_tag
from functools import partial
from itertools import zip_longest
from typing import Union, Tuple, Optional
from fastcore.all import L, delegates
import copy

In [ ]:
from nbdev.showdoc import show_doc
from IPython.display import HTML

## Utils

### Tests

In [ ]:
from fastcore.test import *

In [ ]:
def test_xml_eq(a, b):
    "Tests if if 2 FT components generate the same xml"
    test_eq(*map(lambda x: x if isinstance(x,str) else to_xml(x), (a, b)))

### Other

In [ ]:
#| export
# need a better name, stringify might be too general for what it does 
def stringify(o # String, Tuple, or Enum options we want stringified
             ): # String that can be passed FT comp args (such as `cls=`)
    "Converts input types into strings that can be passed to FT components"  
    if fh.is_listy(o): return ' '.join(map(str,o)) if o else ""
    return o.__str__()

In [ ]:
assert stringify('abc') == 'abc'
assert stringify(('abc','def')) == 'abc def'
assert 'uk-input ' + stringify(()) == 'uk-input '
assert 'uk-input ' + stringify("") == 'uk-input '

In [ ]:
#| export
class VEnum(Enum):
    def __add__(self, other):
        "Add other enums, listy, or strings"
        return stringify((self, other))

    def __radd__(self, other):
        "Add other enums, listy, or strings"
        return stringify((other, self))    
    
    def __str__(self):
        base = self.__class__.__name__.lstrip('Uk').rstrip('T')
        return f"uk-{base.lower()}-{self.value}".strip('-')

In [ ]:
class TestT(VEnum):
    foo = 'foo'
    baz = 'baaz'

assert str(TestT.baz) == 'uk-test-baaz'
assert TestT.baz + TestT.foo + ' bar' == 'uk-test-baaz uk-test-foo bar'
assert stringify((TestT.baz, TestT.foo, 'bar')) == 'uk-test-baaz uk-test-foo bar'

## Theme

You can select a theme color to and get all the headers.  This brings in the most standard grouping:
+ Tailwind: Optional, but typically FrankenUI is used in conjunction with tailwind
+ Uikit: Many things are styled with uikit
+ UIkit icons: Gives many icons OOTB, and is nice to have handy.
+ Franken WC: Neccesary for some of the (minimal) interactivity components have

In [ ]:
#| export
class Theme(Enum):
    slate = "slate"
    stone = "stone"
    gray = "gray"
    neutral = "neutral"
    red = "red"
    rose = "rose"
    orange = "orange"
    green = "green"
    blue = "blue"
    yellow = "yellow"
    violet = "violet"
    zinc = "zinc"

    def headers(self):
        js = (fh.Script(src="https://cdn.tailwindcss.com"),
              fh.Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"),
              fh.Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"),
              fh.Script(type="module", src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js")
              )
        _url = f"https://unpkg.com/franken-wc@0.0.6/dist/css/{self.value}.min.css"
        return (*js, fh.Link(rel="stylesheet", href=_url))

In [ ]:
hdrs = Theme.blue.headers()
app = fh.FastHTML(hdrs=hdrs)

## Text Style

#### `B` (Base)

Base enums all will end with `B`.  These are designed to make classes discoverable in python.  These should be generally a 1:1 mapping to a class.

These are the lowest level class selection options without dropping into strings.  We don't need to cover everything, but we want most useful things in here.

In [ ]:
#| export
class TextB(Enum):
    sz_xsmall = 'text-xs'
    sz_small = 'text-sm'
    sz_medium = 'text-base'
    sz_large = 'text-lg'
    cl_muted = 'uk-text-muted'
    
    wt_light = 'font-light'
    wt_normal = 'font-normal'
    wt_medium = 'font-medium'
    wt_bold = 'font-bold'
    
    def __str__(self): return stringify(self.value)

In [ ]:
print(TextB.sz_xsmall)

text-xs


### `T` (Transform)

Transform enums all will end with `T`.  These are designed to make useful class groups discoverable in python.

Each option should be a group of enums that are useful in many cases.  For example `muted_sm` is very commonly used as a subheader below a card title and looks great.  These should be high level things that users can generally just pick from to have a decent default without much thought.

In [ ]:
#| export
class TextT(Enum):
    muted_xs =  TextB.sz_xsmall, TextB.cl_muted 
    muted_sm =  TextB.sz_small,  TextB.cl_muted # Text below card headings
    muted_med = TextB.sz_medium, TextB.cl_muted 
    muted_lg =  TextB.sz_large,  TextB.cl_muted 
    medium_sm = TextB.sz_small,  TextB.wt_medium
    medium_xs = TextB.sz_xsmall, TextB.wt_medium

    def __str__(self): return stringify(self.value)

In [ ]:
print(TextT.muted_sm)

text-sm uk-text-muted


## Icons and Avatars

In [ ]:
#| export
def Icon(icon,    # Icon name from https://getuikit.com/docs/icon
           ratio=1, # Icon ratio/size 
           cls=()   # Span classes
          ):        # Span with Icon
    "Creates a Span with the given icon"
    return fh.Span(uk_icon=f"icon: {icon}; ratio: {ratio}",cls=stringify(cls))

In [ ]:
Icon('heart', 0.5)

```html
<span uk-icon="icon: heart; ratio: 0.5"></span>
```

In [ ]:
#| export
def DiceBearAvatar(seed_name, # Seed name (ie 'Isaac Flath')
                   h,         # Height 
                   w          # Width
                  ):          # Span with Avatar
    url = 'https://api.dicebear.com/8.x/lorelei/svg?seed='
    return fh.Span(cls=f"relative flex h-{h} w-{w} shrink-0 overflow-hidden rounded-full bg-accent")(
            fh.Img(cls="aspect-square h-full w-full", alt="Avatar", src=f"{url}{seed_name}"))

In [ ]:
DiceBearAvatar('Isaac Flath', 10, 10)

```html
<span class="relative flex h-10 w-10 shrink-0 overflow-hidden rounded-full bg-accent"><img alt="Avatar" src="https://api.dicebear.com/8.x/lorelei/svg?seed=Isaac Flath" class="aspect-square h-full w-full"></span>
```

## Spacing | Flexbox | Grid

The most common pattern for spacing it to organize the general high level page layout with a `Grid`, and smaller components with `Flex`.

:::{.callout-tip}
Play [Flex Box Froggy](https://flexboxfroggy.com/) to get an understanding of flex box.
:::

In [ ]:
#|exports
class FlexT(VEnum):
    block       = ''
    inline      = 'inline'
    #horizontal
    left        = 'left'
    center      = 'center'
    right       = 'right'
    between     = 'between'
    around      = 'around'
    #vertical
    stretch     = 'stretch'
    top         = 'top'
    middle      = 'middle'
    botton      = 'bottom'
    #direction
    row         = 'row'
    row_reverse = 'row-reverse'
    col         = 'col'
    col_reverse = 'col-reverse'
    #wrap
    nowrap      = 'nowrap'
    wrap        = 'wrap'
    wrap_reverse= 'wrap-reverse'

In [ ]:
fh.Div(cls=(FlexT.block,FlexT.left,FlexT.middle,'some_cls'))

```html
<div class="uk-flex uk-flex-left uk-flex-middle some_cls"></div>

```

In [ ]:
#|exports
class GridT(VEnum):
    # gap
    small  = 'small'
    medium = 'medium'
    large  = 'large'
    none   = 'collapse'

In [ ]:
#|export
def Grid(*c,      # Divs/Containers that should be divided into a grid
         cols=3,  # Number of columns
         cls=(),  # Additional classes for Grid Div
         **kwargs # Additional args for Grid Div
        ):
    """Creates a grid with the given number of columns, often used for a grid of cards"""
    cls = stringify(cls)
    return fh.Div(cls=(f'grid grid-cols-{cols}',cls), **kwargs)(*c)

In [ ]:
Grid(*map(fh.Div, range(6)),cols=3, cls=GridT.small)

```html
<div class="grid grid-cols-3 uk-grid-small">
  <div>0</div>
  <div>1</div>
  <div>2</div>
  <div>3</div>
  <div>4</div>
  <div>5</div>
</div>

```

In [ ]:
#| export
def ResponsiveGrid(*c, sm=1, md=2, lg=3, xl=4, gap=2, cls='', **kwargs):
    "Creates a responsive grid with the given number of columns for different screen sizes"
    return fh.Div(cls=f'grid grid-cols-{sm} md:grid-cols-{md} lg:grid-cols-{lg} xl:grid-cols-{xl} gap-{gap} ' + stringify(cls), **kwargs)(*c)

We can use the `FlexT` enum above to create `Div`s that arrange components in a flex box in different ways.  

For example, `FullySpacedDiv` applies a very common pattern so we've made a function that helps to this for you.  But if you look at the code, it's really just using the enum to apply the right combonation of flex classes.

In [ ]:
#| exports
def FullySpacedDiv(*c,                # Components
                   cls='uk-width-1-1',# Classes for outer div
                   **kwargs           # Additional args for outer div
                  ):                  # Div with spaced components via flex classes
    "Creates a flex div with it's components having as much space between them as possible"
    cls = stringify(cls)
    return fh.Div(cls=(FlexT.block,FlexT.between,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
FullySpacedDiv(*map(fh.P, range(3)))

```html
<div class="uk-flex uk-flex-between uk-flex-middle uk-width-1-1">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def CenteredDiv(*c,      # Components
                cls=(),  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components centered in it
    "Creates a flex div with it's components centered in it"
    cls=stringify(cls)
    return fh.Div(cls=(FlexT.block,FlexT.col,FlexT.middle,FlexT.center,cls),**kwargs)(*c)

In [ ]:
CenteredDiv(*map(fh.P, range(3)))

```html
<div class="uk-flex uk-flex-col uk-flex-middle uk-flex-center ">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def LAlignedDiv(*c,      # Components
                cls=(),  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components aligned to the left
    "Creates a flex div with it's components aligned to the left"
    cls=stringify(cls)
    return fh.Div(cls=(FlexT.block,FlexT.left,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
LAlignedDiv(*map(fh.P, range(3)))

```html
<div class="uk-flex uk-flex-left uk-flex-middle ">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def RAlignedDiv(*c,      # Components
                cls=(),  # Classes for outer div
                **kwargs # Additional args for outer div
               ): # Div with components aligned to the right
    "Creates a flex div with it's components aligned to the right"
    cls=stringify(cls)
    return fh.Div(cls=(FlexT.block,FlexT.right,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
RAlignedDiv(*map(fh.P, range(3)))

```html
<div class="uk-flex uk-flex-right uk-flex-middle ">
  <p>0</p>
  <p>1</p>
  <p>2</p>
</div>

```

In [ ]:
#| export
def VStackedDiv(*c, cls='', **kwargs):
    cls=stringify(cls)
    return fh.Div(cls=(FlexT.block,FlexT.col,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
#| export
def HStackedDiv(*c, cls='', **kwargs):
    cls=stringify(cls)
    return fh.Div(cls=(FlexT.block,FlexT.row,FlexT.middle,cls), **kwargs)(*c)

In [ ]:
#| export
def SpaceBetweenDiv(*c, cls='', **kwargs):
    cls = stringify(cls)
    return fh.Div(cls=(FlexT.block,FlexT.between,FlexT.middle,cls), **kwargs)(*c)

## Basic Inputs

Inputs of various types often go with a label.  Because of this we created functions to do this for you along with properly linking the `for` attribute from the lable to the input.  We also have some nice defaults, such as putting a little spacing between the label and the input

In [ ]:
#| export
def GenericInput(input_fn: fh.FT, # FT Components that generates a user input (e.g. `TextArea`)
                    label:str|fh.FT=(), # String or FT component that goes in `Label`
                    lbl_cls:str|Enum=(), # Additional classes that goes in `Label`
                    inp_cls:str|Enum='', # Additional classes that go in user input (e.g. `TextArea`)
                    cls:str|Enum=('space-y-2',), # Div cls
                    id: str="", # ID of the user input (e.g. `TextArea`)
                   **kwargs # Passed to `input_fn` (e.g. ` TextArea`)
                  ) -> fh.FT: # FT component in structure `(Div(label,input))`
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UKTextArea` which is created for you in this library"
    lbl_cls, inp_cls, cls = map(stringify,(lbl_cls, inp_cls, cls))
    if label:  label = fh.Label(cls='uk-form-label '+lbl_cls, fr=id)(label)
    res = input_fn(cls=inp_cls, **kwargs)
    return fh.Div(cls=cls)(label, res)

In [ ]:
#| export
@delegates(GenericInput,but=['input_fn','inp_cls'])
def Input(*args, inp_cls='', **kwargs): 
    "Creates a text input with uk styling"
    inp_cls = ('uk-input',stringify(inp_cls))
    return GenericInput(fh.Input, *args, inp_cls=inp_cls, **kwargs)

In [ ]:
Input('myLabel', id='my_id')

```html
<div class="space-y-2">
<label for="my_id" class="uk-form-label ">myLabel</label>  <input class="uk-input ">
</div>

```

In [ ]:
#|export
@delegates(GenericInput,but=['input_fn','inp_cls'])
def Switch(*args, inp_cls='', **kwargs): 
    "Creates a switch input with uk styling"
    inp_cls = ('uk-toggle-switch uk-form-switch',stringify(inp_cls))
    return GenericInput(fh.CheckboxX, *args, inp_cls=inp_cls, **kwargs)

In [ ]:
Switch(cls='')

```html
<div>
  <input type="hidden" skip>
  <input type="checkbox" value="1" class="uk-toggle-switch uk-form-switch ">
</div>

```

In [ ]:
#|export
@delegates(GenericInput,but=['input_fn','inp_cls'])
def Checkbox(*args, inp_cls='', **kwargs): 
    "Creates a checkbox input with uk styling"
    inp_cls = ('uk-checkbox',stringify(inp_cls))
    return GenericInput(fh.CheckboxX, *args, inp_cls=inp_cls, **kwargs)

In [ ]:
Checkbox('myLabel', inp_cls='another_class')

```html
<div class="space-y-2">
<label class="uk-form-label ">myLabel</label>  <input type="hidden" skip>
  <input type="checkbox" value="1" class="uk-checkbox another_class">
</div>

```

In [ ]:
#|export
@delegates(GenericInput,but=['input_fn','inp_cls'])
def TextArea(*args, inp_cls='', **kwargs): 
    "Creates a textarea with uk styling"
    inp_cls = ('uk-textarea',stringify(inp_cls))
    return GenericInput(fh.Textarea, *args, inp_cls=inp_cls,  **kwargs)

In [ ]:
TextArea('myLabel', lbl_cls='another_class')

```html
<div class="space-y-2">
<label class="uk-form-label another_class">myLabel</label><textarea class="uk-textarea "></textarea></div>

```

In [ ]:
#|export
@delegates(GenericInput,but=['input_fn','inp_cls'])
def FormLabel(*args, inp_cls='', **kwargs): 
    "Creates a form label with uk styling"
    inp_cls = ('uk-form-label',stringify(inp_cls))
    return GenericInput(Uk_input_tag ,*args, inp_cls=inp_cls, **kwargs)

In [ ]:
FormLabel('myLabel', lbl_cls=TextT.muted_sm)

```html
<div class="space-y-2">
<label class="uk-form-label text-sm uk-text-muted">myLabel</label><uk-input-tag class="uk-form-label "></uk-input-tag></div>

```

## Buttons

In [ ]:
#| export
class ButtonT(VEnum):
    default   = 'default'
    primary   = 'primary'
    secondary = 'secondary'
    danger    = 'danger'
    ghost     = 'ghost'
    text      = 'text'
    link      = 'link'

In [ ]:
#| export
def Button(*c, # Content for the button
            cls=ButtonT.default, # Classes for the button
            **kwargs # Additional args for the button
            ):
    "Creates a button with uk styling"
    cls = stringify(cls)    
    return fh.Button(type='button', cls=('uk-button',cls), **kwargs)(*c)

In [ ]:
Button('MyButton', cls=UkButtonT.primary)

```html
<button type="button" class="uk-button uk-button-primary">MyButton</button>
```

In [ ]:
#| export
def IconButton(*c, sz='small', cls=(), **kwargs):
    "Creates an `IconButton` with uk styling"
    if sz not in ('small','medium','large'): raise ValueError(f"Invalid size '{sz}'. Must be 'small', 'medium', or 'large'.")
    return fh.Button(cls=f'uk-icon-button uk-icon-button-{sz} ' + stringify(cls), **kwargs)(*c)

In [ ]:
IconButton(Icon('home'))

```html
<button class="uk-icon-button uk-icon-button-small "><span uk-icon="icon: home; ratio: 1"></span></button>
```

## Drop Downs

In [ ]:
#| export
def Options(*c,                    # Content for an `Option`
            selected_idx:int=None, # Index location of selected `Option`
            disabled_idxs:set=None # Idex locations of disabled `Options`
           ):
    "Helper function to wrap things into `Option`s for use in `UKSelect`"
    return [fh.Option(o,selected=i==selected_idx, disabled=disabled_idxs and i in disabled_idxs) for i,o in enumerate(c)]

In [ ]:
list(map(fh.to_xml,Options('option1','option2','option3', selected_idx=1, disabled_idxs={0,2})))

['<option disabled>option1</option>',
 '<option selected>option2</option>',
 '<option disabled>option3</option>']

In [ ]:
#| export
def Select(*option,            # Options for the select dropdown (can use `Options` helper function to create)
             label=(),           # String or FT component for the label
             lbl_cls=(),         # Additional classes for the label
             inp_cls=(),         # Additional classes for the select input
             cls=('space-y-2',), # Classes for the outer div
             id="",              # ID for the select input
             name="",            # Name attribute for the select input
             placeholder="",     # Placeholder text for the select input
             searchable=False,   # Whether the select should be searchable
             **kwargs):          # Additional arguments passed to Uk_select
    "Creates a select dropdown with uk styling"
    lbl_cls, inp_cls, cls = map(stringify, (lbl_cls, inp_cls, cls))
    if label: 
        lbl = fh.Label(cls=f'uk-form-label {lbl_cls}', fr=id)(label) 
    select = Uk_select(*option, cls=inp_cls, uk_cloak=True, id=id, 
                       name=name, placeholder=placeholder, searchable=searchable, **kwargs)
    return fh.Div(cls=cls)(lbl, select) if label else fh.Div(cls=cls)(select)

In [ ]:
Select(*Options('a', 'b'), label='MyLabel', placeholder='MyPlaceholder')

```html
<div class="space-y-2">
<label class="uk-form-label ">MyLabel</label><uk-select uk-cloak placeholder="MyPlaceholder"><option>a</option><option>b</option></uk-select></div>

```

In [ ]:
#| exporti
def _DropdownButtonOptions(opt_grps, opt_hdrs=None):
    res = []
    for g,h in zip_longest(opt_grps, fh.tuplify(opt_hdrs)):
        if h: res.append(fh.Li(cls="uk-nav-header")(h if isinstance(h,fh.FT) else fh.Div(h)))
        if isinstance(g,(list,tuple)): res += list(map(fh.Li, g))
        else: res.append(fh.Li(g))
    return res

In [ ]:
#| export
def DropdownButton(
    *opt_grp,        # List of options to be displayed in the dropdown
    opt_hdrs=None,  # List of headers for each option group, or None
    label=None,     # String, FT component, or None for the `Button`
    btn_cls=ButtonT.default,  # Button class(es)
    cls=(),         # Parent div class
    dd_cls=(),      # Class that goes on the dropdown container
    icon='triangle-down',  # Icon to use for the dropdown
    icon_cls='',    # Additional classes for the icon
    icon_position='right'  # Position of the icon: 'left' or 'right'
    ):
    dd_cls, btn_cls, cls, icon_cls = map(stringify, (dd_cls, btn_cls, cls, icon_cls))
    icon_component = Icon(icon, cls=icon_cls) if icon else None
    btn_content = [] if label is None else [label]
    if icon_component: btn_content.insert(0 if icon_position == 'left' else len(btn_content), icon_component)
    btn = fh.Button(type='button', cls='uk-button ' + btn_cls)(*btn_content)
    dd = fh.Div(uk_dropdown='mode: click; pos: bottom-right', cls='uk-dropdown uk-drop ' + dd_cls)(
        fh.Ul(cls='uk-dropdown-nav')(*_DropdownButtonOptions(opt_grp, opt_hdrs)))
    return fh.Div(cls=cls)(fh.Div(cls='flex items-center space-x-4')(btn, dd))

In [ ]:
# DropdownButton((fh.A('Option 1.1', _href='#'), fh.A('Option 1.2', _href='#')), 
#                  (fh.A('Option 2.1', _href='#'), fh.A('Option 2.2', _href='#')),
#                  opt_hdrs=('Header 1', 'Header 2'))

## Headings

In [ ]:
#| export
def GenericComponent(component_fn, *c, cls=(), **kwargs):
    res = component_fn(cls=cls, **kwargs)(*c)
    return res

In [ ]:
#| export
def H1(*c, cls=(), **kwargs): return GenericComponent(fh.H1, *c, cls=('uk-h1',stringify(cls)), **kwargs)
def H2(*c, cls=(), **kwargs): return GenericComponent(fh.H2, *c, cls=('uk-h2',stringify(cls)), **kwargs)
def H3(*c, cls=(), **kwargs): return GenericComponent(fh.H3, *c, cls=('uk-h3',stringify(cls)), **kwargs)
def H4(*c, cls=(), **kwargs): return GenericComponent(fh.H4, *c, cls=('uk-h4',stringify(cls)), **kwargs)
def H5(*c, cls=(), **kwargs): return GenericComponent(fh.H5, *c, cls=('uk-h5',stringify(cls)), **kwargs)
def H6(*c, cls=(), **kwargs): return GenericComponent(fh.H6, *c, cls=('uk-h6',stringify(cls)), **kwargs)

In [ ]:
H1('My Uk styled header',cls='another-class')

```html
<h1 class="uk-h1 another-class">My Uk styled header</h1>

```

## Dividers

In [ ]:
#| export
def HSplit(*c, cls=(), line_cls=(), text_cls=()):
    cls, line_cls, text_cls = map(stringify,(cls, line_cls, text_cls))
    return fh.Div(cls='relative ' + cls)(
        fh.Div(cls="absolute inset-0 flex items-center " + line_cls)(fh.Span(cls="w-full border-t border-border")),
        fh.Div(cls="relative flex justify-center " + text_cls)(fh.Span(cls="bg-background px-2 ")(*c)))

In [ ]:
# HSplit()

In [ ]:
#| export
def HLine(lwidth=2, y_space=4): return fh.Div(cls=f"my-{y_space} h-[{lwidth}px] w-full bg-secondary")

In [ ]:
HLine()

```html
<div class="my-4 h-[2px] w-full bg-secondary"></div>

```

In [ ]:
#| export
def NavDivider(): return fh.Li(cls="uk-nav-divider")

In [ ]:
NavDivider()

```html
<li class="uk-nav-divider"></li>

```

## NavBar

In [ ]:
#| export
def NavbarDropdown(*c, label, href='#', cls='', has_header=False, **kwargs):
    fn = lambda x: fh.Li(item, cls='uk-drop-close', href='#demo', uk_toggle=True)
    flattened = []
    for i, item in enumerate(c):
        if i > 0: flattened.append(fh.Li(cls="uk-nav-divider"))
        if isinstance(item, (list,tuple)): flattened.extend(map(fh.Li, item))
        else: flattened.append(fh.Li(item, cls="uk-nav-header" if i == 0 and has_header else None, uk_toggle=True))
    return (fh.Li(cls=cls, **kwargs)(
                fh.A(label, cls='uk-drop-close', href='#', uk_toggle=True), 
                fh.Div(cls='uk-navbar-dropdown', uk_dropdown="mode: click; pos: bottom-left")(fh.Ul(cls='uk-nav uk-dropdown-nav')(*flattened))))

In [ ]:
#| export
def _NavBarSide(n, s):
    def add_class(item):
        if isinstance(item, str): return fh.Li(cls='uk-navbar-item')(item)
        else: item.attrs['class'] = f"{item.attrs.get('class', '')} uk-navbar-item".strip()
        return item
    return fh.Div(cls=f'uk-navbar-{s}')(fh.Ul(cls='uk-navbar-nav')(*map(add_class, fh.tuplify(n))))

In [ ]:
#| export
def Navbar(lnav: fh.Sequence[Union[str, fh.FT]]=None, 
             rnav: fh.Sequence[Union[str, fh.FT]]=None, 
             cls='',
             **kwargs
            ) -> fh.FT:
    return fh.Div(cls='uk-navbar-container uk-width-1-1 relative z-10 '+ stringify(cls), uk_navbar=True, **kwargs)(
             _NavBarSide(lnav,'left') if lnav else '',
             _NavBarSide(rnav,'right') if rnav else '')

## Sidebar

In [ ]:
sidebar_items = ["Profile", "Account", "Appearance", "Notifications", "Display"]
tst = fh.Ul(cls='uk-nav-primary', uk_switcher="connect: #component-nav; animation: uk-animation-fade")(
                            *map(lambda x: fh.Li(fh.A(x)),sidebar_items))

In [ ]:
#| export
def Sidebar(*ul,                 # Each Ul can be it's own section.  Use A for links!
              cls='space-y-4 p-4', # Classes for outer container
              **kwargs             # Kwargs for outer container
             ):
    "Creates a styled sidebar component"
    styles = ('uk-nav-default', 'uk-nav-primary','uk-nav-secondary')
    sidebar = []
    for section in fh.tuplify(ul):
        section = copy.deepcopy(section)
        _sattrs = section.attrs
        if 'class' not in _sattrs: _sattrs['class'] = ''
        if 'uk-nav' not in _sattrs: _sattrs['class'] += ' uk-nav '
        if not any(x in styles for x in _sattrs['class'].split()): _sattrs['class'] += ' uk-nav-default '
        sidebar.append(section)  
    return fh.Div(cls=cls, **kwargs)(*sidebar)

In [ ]:
sidebar_items = ["Profile", "Account"]
tst = fh.Ul(cls='uk-nav-primary')(*map(lambda x: fh.Li(fh.A(x)),sidebar_items))
Sidebar(tst)

```html
<div class="space-y-4 p-4">
  <ul class="uk-nav-primary uk-nav ">
    <li>
<a href="#">Profile</a>    </li>
    <li>
<a href="#">Account</a>    </li>
  </ul>
</div>

```

## Navtab

In [ ]:
#| export
def NavTab(text, active=False):
    return fh.Li(cls="uk-active" if active else " ")(fh.A(text, href="#demo", uk_toggle=True))

def Tab(*items, maxw=96, cls='', **kwargs):
    cls = stringify(cls)
    return fh.Ul(cls=f"uk-tab-alt max-w-{maxw} "+cls,**kwargs)(*[NavTab(item, active=i==0) for i, item in enumerate(items)])

## Cards

In [ ]:
#|export
def Card(*c, # Components that go in the body
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls=(), # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [fh.Div(cls='uk-card-header ' + header_cls)(header),]
    res += [fh.Div(cls='uk-card-body ' + body_cls)(*c),]
    if footer: res += [fh.Div(cls='uk-card-footer ' + footer_cls)(footer),]
    return fh.Div(cls='uk-card '+cls, **kwargs)(*res)

## Modals

In [ ]:
#| export
def ModalTitle(*c, cls=()): return fh.Div(cls='uk-modal-title ' + stringify(cls))(*c)

def Modal(*c,
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls='p-6', # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [fh.Div(cls='uk-modal-header ' + header_cls)(header),]
    res += [fh.Div(cls='uk-modal-body uk-modal-dialog ' + body_cls)(*c),]
    if footer: res += [fh.Div(cls='uk-modal-footer ' + footer_cls)(footer),]
    return fh.Div(cls='uk-modal uk-modal-container' + cls, uk_modal=True, **kwargs)(*res)


## Tables

In [ ]:
#| export
def _default_cell(col, row): return fh.Td(row[col], cls='p-2')

In [ ]:
def render_header(col):
    cls = 'p-2 ' + 'uk-table-shrink' if col in ('Done','Actions') else ''
    return fh.Th(col, cls=cls)

def render_cell(col, row):
    match col:
        case "ColA":  return fh.Td(row[col], cls=TextT.muted_sm)
        case "ColB":  return fh.Td(row[col], cls=TextT.medium_sm)
        case 'ColC':  return fh.Td(row[col])
        case 'ColD':  return fh.Td(row[col], cls='p-2')
        
cols = ('ColA', 'ColB', 'ColC', 'ColD')
data = [{'ColA':1, 'ColB':2, 'ColC':3, 'ColD':4},]
Table(cols, data, cell_render=render_cell, header_render=render_header)

TypeError: Table() missing 2 required positional arguments: 'columns' and 'data'

In [ ]:
#| export
def TableHeader(columns, header_render=None):
    rndr = header_render or fh.Th(cls='p-2')
    return fh.Tr(*map(rndr, columns))

In [ ]:
#| export
def TableRow(row, columns, cell_render=None):
    rndr = cell_render or _default_cell
    return fh.Tr(*[rndr(col, row) for col in columns])

In [ ]:
#| export
def Table(columns, data, *args, cls=(), footer=None, cell_render=None, header_render=None, **kwargs):
    # table middle, small : Should these be parameterized?
    # Document, especially cell and header render
    table_cls = 'uk-table uk-table-middle uk-table-divider uk-table-hover uk-table-small ' + stringify(cls)
    cell_render   = fh.ifnone(cell_render,   lambda c,r: fh.Td(cls='p-2')(r[c]))
    header_render = fh.ifnone(header_render, lambda c:   fh.Th(cls='p-2')(c))
    
    head = fh.Thead(fh.Th(*map(header_render, columns)))
    
    table_content = []
    for row in data: table_content.append(fh.Tr(*[cell_render(col, row) for col in columns]))
        
    body = fh.Tbody(*table_content)
    if footer: table_content.append(fh.Tfoot(footer))
    return fh.Table(cls=table_cls, *args, **kwargs)(*[head,body])

## Forms

In [ ]:
#| export
def FormSection(title, description, *c, button_txt='Update', outer_margin=6, inner_margin=6):
    return fh.Div(cls=f'space-y-{inner_margin} py-{outer_margin}')(
        fh.Div(H3(title), fh.P(description, cls=TextT.medium_sm)),
        HSplit(), *c,
        fh.Div(Button(button_txt, cls=ButtonT.primary)) if button_txt else None)

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()